# 네이버 리뷰 데이터를 활용한 한국어 감정 분석
네이버 영화 리뷰데이터(Naver Sentiment Movie Corpus,NSMC)를 활용해서 감정분석을 수행했습니다.  
.   
1)
전처리 코드는 다음 github에서 가져와서 사용했습니다.
출처 : https://github.com/reniew/NSMC_Sentimental-Analysis/blob/master/notebook/NSMC_Preprocessing.ipynb  
.  
2)
모델링은 IMDB에서 수행한 LSTM 으로 진행했습니다.


In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 47.5 MB/s eta 0:00:00


In [ ]:
import os

import numpy as np
import pandas as pd

from datetime import datetime
import json
import re

from konlpy.tag import Okt # komoran, han, kkma

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

from tqdm.notebook import tqdm

## 데이터 불러오기

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', header=0, delimiter='\t' ,quoting=3)
test = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', header=0, delimiter='\t' ,quoting=3)

In [ ]:
display(train.head())
display(test.head())

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
train.shape, test.shape

((150000, 3), (50000, 3))

## 데이터 전처리

### Implementation 1

In [ ]:
def preprocessing(review, okt, remove_stopwords = False, stop_words = []):
    # 함수의 인자는 다음과 같다.
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 생성하지 않고 미리 생성후 인자로 받는다.
    # remove_stopword : 불용어를 제거할지 선택 기본값은 False
    # stop_word : 불용어 사전은 사용자가 직접 입력해야함 기본값은 비어있는 리스트

    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    word_review = re.sub('[^가-힣\s]', '', review)

    # 2. okt 객체를 활용해서 형태소 단위(품사까지 반환)로 나눈다.
    word_review = okt.pos(word_review, stem=True)

    # 3. 불용어 제거
    # word_review = [(token, pos) for token, pos in word_review if not token in stop_words](

    # 4. 노이즈 제거 (글자 1개를 노이즈로 간주하고 제거하도록 하자.)
    # word_review = [(token, pos) for token, pos in word_review if len(token) > 1]

    word_review = [(token, pos) for token, pos in word_review if not token in stop_words and (len(token) > 1)]

    # 5. 명사, 동사, 형용사만 추출(품사는 제외하고, 토큰만 반환) : Noun, Verb, Adjective
    word_review = [token for token, pos in word_review if pos in ['Noun', 'Verb', 'Adjective']]

    return word_review

In [ ]:
sample_review = train['document'][0]
sample_review

'아 더빙.. 진짜 짜증나네요 목소리'

In [ ]:
# stop_words = [ '은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']
stop_words = ['목소리']
preprocessing(sample_review, Okt(), remove_stopwords = True, stop_words=stop_words)

['더빙', '진짜', '짜증나다']

In [ ]:
stop_words = ['목소리']
okt = Okt()
clean_review = []
clean_review_test = []

# for문을 apply로 바꿔보기
for review in tqdm(train['document']):
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(review) == str:
        clean_review.append(preprocessing(review, okt, remove_stopwords = True, stop_words=stop_words))
    else:
        clean_review.append([])

for review in tqdm(test['document']):
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(review) == str:
        clean_review_test.append(preprocessing(review, okt, remove_stopwords = True, stop_words=stop_words))
    else:
        clean_review_test.append([])

  0%|          | 0/150000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
clean_review

[['더빙', '진짜', '짜증나다'],
 ['포스터', '보고', '초딩', '영화', '오버', '연기', '가볍다', '않다'],
 ['무재', '밓었', '다그', '래서', '보다', '추천'],
 ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정'],
 ['몬페',
  '익살스럽다',
  '연기',
  '돋보이다',
  '영화',
  '스파이더맨',
  '늙다',
  '보이다',
  '하다',
  '커스틴',
  '던스트',
  '이쁘다',
  '보이다'],
 ['걸음', '떼다', '초등학교', '학년', '생인', '살다', '영화', '반개', '아깝다'],
 ['원작', '긴장감', '제대로', '살리다', '하다'],
 ['반개',
  '아깝다',
  '나오다',
  '이응경',
  '길용우',
  '생활',
  '정말',
  '해도',
  '그것',
  '낫다',
  '납치',
  '감금',
  '반복',
  '반복',
  '드라마',
  '가족',
  '없다',
  '연기',
  '하다',
  '사람'],
 ['액션', '없다', '재미', '있다', '안되다', '영화'],
 ['왜케', '평점', '낮다', '헐리우드', '화려하다', '길들이다', '있다'],
 ['인피니트', '진짜'],
 ['볼때', '눈물나다', '죽다', '년대', '향수', '허진호', '감성', '절제', '멜로', '달인'],
 ['울면', '횡단보도', '건너다', '뛰다', '치다', '이범수', '연기', '드럽다', '하다'],
 ['담백하다', '깔끔하다', '좋다', '로만', '보다', '보다', '자꾸', '잊어버리다', '사람', '이다'],
 ['취향',
  '존중',
  '다지',
  '진짜',
  '내생',
  '극장',
  '보다',
  '영화',
  '가장',
  '노잼',
  '감동',
  '스토리',
  '어거지',
  '감동',
  '어거지'],
 ['매번', '긴장', '되다', 

In [ ]:
clean_review_test

[['굳다'],
 [],
 ['평점', '나쁘다', '않다', '짜다', '더욱', '아니다'],
 ['지루하다', '않다', '완전', '막장', '주다', '보기'],
 ['아니다', '다섯', '주다', '나오다', '심기', '불편하다', '하다'],
 ['음악', '주가', '되다', '최고', '음악', '영화'],
 ['진정하다', '쓰레기'],
 ['마치', '미국', '애니', '튀어나오다', '창의력', '없다', '로봇', '디자인', '고개', '젖다', '하다'],
 ['갈수록',
  '개판',
  '되다',
  '중국영화',
  '유치하다',
  '내용',
  '없다',
  '잡다',
  '끝나다',
  '안되다',
  '무기',
  '유치하다',
  '남무',
  '그리다',
  '동사서독',
  '같다',
  '영화',
  '이건',
  '류작'],
 ['이별', '아픔', '찾아오다', '새롭다', '인연', '기쁨', '모든', '사람', '그렇다', '않다'],
 ['괜찮다', '포켓몬스터', '잼밌'],
 ['한국', '독립영화', '한계', '아버지', '되다', '비교', '되다'],
 ['청춘',
  '아름답다',
  '아름답다',
  '이성',
  '흔들다',
  '놓다',
  '찰나',
  '아름답다',
  '자다',
  '포착',
  '섬세하다',
  '아름답다',
  '수채화',
  '같다',
  '퀴어',
  '영화'],
 ['보이다', '반전', '이다', '영화', '흡인', '사라지다', '않다'],
 ['스토리',
  '연출',
  '연기',
  '비주',
  '영화',
  '기본',
  '조차',
  '안되다',
  '영화',
  '무슨',
  '이렇다',
  '영화',
  '찍다',
  '김문옥',
  '감독',
  '영화',
  '경력',
  '조무래기',
  '영화',
  '평론',
  '하다',
  '같다',
  '마인드',
  '빠지다'],
 ['소위', '가라', '평점'],
 ['최고'],

In [ ]:
# pickle 파일로 저장하기
# import pickle

# with open('./clean_review_test.pickle', 'wb') as f:
#   pickle.dump(clean_review_test, f)

In [ ]:
# pickle 파일 불러오기
# import pickle

# with open('./clean_review.pickle', 'rb') as f:
#   clean_review = pickle.load(f)

# with open('./clean_review_test.pickle', 'rb') as f:
#   clean_review_test = pickle.load(f)

In [ ]:
print(len(clean_review))
print(len(clean_review_test))

150000
50000


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_review) # 단어 인덱스 구축
text_sequences = tokenizer.texts_to_sequences(clean_review) # 문자열 -> 인덱스 리스트
                                                            # '나는 천재다 나는 멋있다' -> [1, 2, 1, 3]
# train data로 fit 시켜준 tokenizer로 test data도 인덱스로 변경
text_sequences_test = tokenizer.texts_to_sequences(clean_review_test)

word_vocab = tokenizer.word_index # 딕셔너리 형태
print("전체 단어 개수: ", len(word_vocab)) # 전체 단어 개수 확인

전체 단어 개수:  40542


In [ ]:
MAX_SEQUENCE_LENGTH = 30 # 문장 최대 길이

X_train = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 50 단어가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_train = np.array(train['label']) # 각 리뷰의 감정을 넘파이 배열로 만든다.

print('Shape of input data tensor:', X_train.shape) # 리뷰 데이터의 형태 확인
print('Shape of label tensor:', y_train.shape) # 감정 데이터 형태 확인

Shape of input data tensor: (150000, 30)
Shape of label tensor: (150000,)


In [ ]:
MAX_SEQUENCE_LENGTH = 30 # 문장 최대 길이
X_test = pad_sequences(text_sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 50 단어가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_test = np.array(test['label']) # 각 리뷰의 감정을 넘파이 배열로 만든다.

print('Shape of input data tensor:', X_test.shape) # 리뷰 데이터의 형태 확인
print('Shape of label tensor:', y_test.shape) # 감정 데이터 형태 확인

Shape of input data tensor: (50000, 30)
Shape of label tensor: (50000,)


In [ ]:
X_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0, 289,  11, 152], dtype=int32)

## 모델 구축

### Implementation 2

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

model = Sequential()
model.add(Embedding(len(word_vocab)+1, 100)) # (단어집합의 크기, 임베딩 후 벡터 크기)
model.add(LSTM(128))
model.add(Dense(1, activation = 'sigmoid')) # 0 or 1로 이진분류이므로 시그모이드 함수를 사용
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics=['accuracy'])
# 이진 분류이므로 손실함수는 binary_crossentropy 사용, 에폭마다 정확도를 보기 위해 accuracy 적용
print(model.summary()) #모델 아키텍처 출력

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         4054300   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 4,171,677
Trainable params: 4,171,677
Non-trainable params: 0
_________________________________________________________________
None


## 모델 학습

### Implementation 3

In [ ]:
## model fit
model.fit(X_train, y_train, epochs=7, batch_size=64, validation_split=0.2)

Epoch 1/7
1875/1875 [==============================] - 52s 23ms/step - loss: 0.4317 - accuracy: 0.7927 - val_loss: 0.4008 - val_accuracy: 0.8155
Epoch 2/7
1875/1875 [==============================] - 14s 8ms/step - loss: 0.3735 - accuracy: 0.8313 - val_loss: 0.3866 - val_accuracy: 0.8244
Epoch 3/7
1875/1875 [==============================] - 12s 7ms/step - loss: 0.3508 - accuracy: 0.8437 - val_loss: 0.3794 - val_accuracy: 0.8290
Epoch 4/7
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3287 - accuracy: 0.8565 - val_loss: 0.3711 - val_accuracy: 0.8326
Epoch 5/7
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3066 - accuracy: 0.8700 - val_loss: 0.3705 - val_accuracy: 0.8324
Epoch 6/7
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2857 - accuracy: 0.8812 - val_loss: 0.3852 - val_accuracy: 0.8303
Epoch 7/7
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2667 - accuracy: 0.8892 - val_loss: 0.3990 - val_accuracy

## 모델 검증

In [ ]:
from sklearn.metrics import accuracy_score

y_train_predclass = model.predict(X_train)
y_test_predclass = model.predict(X_test)

y_train_prediction = np.where(y_train_predclass > 0.5, 1, 0)
y_test_prediction = np.where(y_test_predclass > 0.5, 1, 0)

print("Train Accuracy: {}".format(round(accuracy_score(y_train, y_train_prediction),3)))
print("Test Accuracy: {}".format(round(accuracy_score(y_test, y_test_prediction),3)))

1563/1563 [==============================] - 4s 3ms/step
Train Accuracy: 0.887
Test Accuracy: 0.826


## 네이버웹툰 댓글에 감성라벨링하기

In [ ]:
!pip install --upgrade --no-cache-dir gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6


In [ ]:
!gdown https://drive.google.com/uc?id=1s4w81tICR74bXPXGww6DNA6Xs6WtOZeX

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1s4w81tICR74bXPXGww6DNA6Xs6WtOZeX 



#### 웹툰 댓글 직접 크롤링

In [ ]:
import requests
import json

In [ ]:
# 96화
'objectId':'783053_96'
# 95화 비교
objectId=783053_95

In [ ]:
def get_toon_all_comment_page(no=1, page=1):
    url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json'

    params = {'ticket':'comic',
            'templateId':'webtoon',
            'pool':'cbox3',
            '_cv':'20230829152227',
            '_callback':'jQuery36308930865526982845_1693788518199',
            'lang':'ko',
            'country':'KR',
            'objectId':f'783053_{no}',
            'pageSize':15,
            'indexSize':10,
            'groupId':783053,
            'listType':'OBJECT',
            'pageType':'more',
            'page':page,
            'currentPage':1,
            'refresh':'true',
            'sort':'new'}

    headers = {
        'Referer':'https://comic.naver.com/webtoon/detail?titleId=783053&no=96',
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    }

    resp = requests.get(url, params=params, headers=headers)
    return resp

In [ ]:
resp = get_toon_all_comment_page(page=1)
resp.text

'jQuery36308930865526982845_1693788518199({"success":true,"code":"1000","message":"요청을 성공적으로 처리하였습니다.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"comic","objectId":"783053_1","categoryId":"*","templateId":"webtoon","commentNo":"458597522","parentCommentNo":"458597522","replyLevel":1,"replyCount":0,"replyAllCount":0,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1693575297570,"contents":"민지인지 머시깽인지도 같은 애들 같은데","userIdNo":"1uVj5","exposedUserIp":null,"lang":"ko","country":"KR","idType":"naver","idProvider":"naver","userName":"차좀빼줘","userProfileImage":null,"profileType":"comic","modTime":"2023-09-01T22:34:58+0900","modTimeGmt":"2023-09-01T13:34:58+0000","regTime":"2023-09-01T22:34:58+0900","regTimeGmt":"2023-09-01T13:34:58+0000","sympathyCount":0,"antipathyCount":0,"hideReplyButton":false,"status":0,"mine":false,"best

In [ ]:
# 댓글 : result commentList [] contents
len('jQuery36308930865526982845_1693788518199(')

41

In [ ]:
resp_dic = json.loads(resp.text[41:-2])
commentList = resp_dic['result']['commentList']
for comment in commentList:
    print(comment['contents'])

민지인지 머시깽인지도 같은 애들 같은데
그럼 뭐하노 딸은 이미 죽었는데 있을때 잘해야지
박태준이 그린거 아닌데 뭔 박태준 만화야 걍 사장일뿐
와 처음보는데 스릴넘치고 
사이다 58000L콸콸 쏟아지네
김부장? ㅋㅋ 그 김부장??
민지 어디있어하고 광고에 뉴진스민지 나오네
어이 아저씨 좀 쳐?
???: 응 쥬우우운나
회사에서 부장다는게 얼마나 어려운데!!!!
아무리 화나도 그렇지 아빠한테 어뜨케 그렇게 말하냐.. 속상하다 ㅠㅠ 
초반에 남실장이네
외지주 
연출콘티 갸오오 뭔데 ㅋㅋ
박태준 만화회사는 다 재밋다
중앙정보부 부장이면 대기업 회장도 군대 이끌고 잡아다 끌고 고문 할 수 있는데ㅋㅋㅋㅋ
대한민국에선 능력없으면 아이낳는거 자체가 죄임


In [ ]:
for page in range(1, 25):
    resp = get_toon_all_comment_page(page=page)
    resp_dic = json.loads(resp.text[41:-2])
    commentList = resp_dic['result']['commentList']
    for comment in commentList:
        print(comment['contents'])

민지인지 머시깽인지도 같은 애들 같은데
그럼 뭐하노 딸은 이미 죽었는데 있을때 잘해야지
박태준이 그린거 아닌데 뭔 박태준 만화야 걍 사장일뿐
와 처음보는데 스릴넘치고 
사이다 58000L콸콸 쏟아지네
김부장? ㅋㅋ 그 김부장??
민지 어디있어하고 광고에 뉴진스민지 나오네
어이 아저씨 좀 쳐?
???: 응 쥬우우운나
회사에서 부장다는게 얼마나 어려운데!!!!
아무리 화나도 그렇지 아빠한테 어뜨케 그렇게 말하냐.. 속상하다 ㅠㅠ 
초반에 남실장이네
외지주 
연출콘티 갸오오 뭔데 ㅋㅋ
박태준 만화회사는 다 재밋다
중앙정보부 부장이면 대기업 회장도 군대 이끌고 잡아다 끌고 고문 할 수 있는데ㅋㅋㅋㅋ
대한민국에선 능력없으면 아이낳는거 자체가 죄임
응당일 흔들어봐 생각나네 
뭔 대기업 부장도 능력없는 인간이란 소릴하고 있네 ㅋㅋ 작가가 미친거아님?
그림체가 예전 대가리 작가님 같으신데... ?ㅋ
이땐 몰랐습니다 아빠라는 사람이 이렇게 위대한줄은.
테이큰이갘ㅋㅋㅋㅋㅋ
김부장 이름이 김부장이에요?
โคตรมัน
근데 친구 아빠들이 국회의원에 뭐 어디 사장인거면 무슨 스카이캐슬이냐
지 아빠가 부장정도만 해도 감사할줄 알아야지 자식스키나 돼서 말이야 ㅉ
박태준 욕하는 애들 특) 능력없음
히히
이거 확 조져버려?
이거 만든사람 당장 나와
이거 내 옛날 상화극이니까 
너 누가 이거 만들래?
누가?
만들어 된다 했니?
이 양심도 없니
그리고 김부장이 남자가
이렇게 남자야?
나같으면 조진다에 내목숨건다
그리고 김부장은 여자냐?
남자가 왜 이렇게 겁이 많냐?
그리고 나중에 민지를 찾으면서 지랄하잖아
근디 왜
그때는 왜 지랄않했을까?
그딴거 만들꺼면 
나라에 허가받고 그리든가 해야 정상아니냐?
지금 국회의원.아들땜에
난린데
이거 외지주 김부장이죠?
우리아빤 직업도 없는데 잘만 살아간다..이눔아
진심 웹툰인데도 개빡치네
딸이 ㅎㅂㄴ이네 잘뒈짐
와..존잼작 찾았다 
너다
이거  청소부 웹툰내용과똑같은데 ㅡㅡ
아 대가리 작가구나 근데 솔직히 액션신 그거 표절하나로 인생 조지는거도 웃긴데 난 ㅋ

In [ ]:
import pandas as pd
df_comment = pd.read_csv("네이버웹툰_댓글스크랩핑결과.csv")
df_comment

### Implementation 4

- preprocessing 함수 이용하여 전처리
- text_to_sequences()함수 이용하여 벡터화
- pad_sequence()함수 이용하여 벡터 크기 맞추기

In [ ]:
## preprocessing 함수 이용하여 전처리
clean_contents = []

okt = Okt()
for review in (comment['contents']):
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(review) == str:
        clean_contents.append(preprocessing(review, okt, remove_stopwords = True, stop_words=stop_words))
    else:
        clean_contents.append([])
## text_to_sequences()함수 이용하여 벡터화

tokenizer.fit_on_texts(clean_contents)
new_text_sequences = tokenizer.texts_to_sequences(clean_contents) # 문자열 -> 인덱스 리스트
                                                            # '나는 천재다 나는 멋있다' -> [1, 2, 1, 3]

## pad_sequence()함수 이용하여 벡터 크기 맞추기
MAX_SEQUENCE_LENGTH = 30 # 문장 최대 길이

new_text = pad_sequences(new_text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post') # 문장의 길이가 50 단어가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.

print('Shape of input data tensor:', new_text.shape) # 리뷰 데이터의 형태 확인



In [ ]:
pred = model.predict(new_text)
pred_label = np.where(pred > 0.5, 1, 0)
pred_label

array([[0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
    

In [ ]:
df_comment['pred_label'] = pred_label
df_comment[['contents', 'pred_label']].tail(10)

,contents,pred_label
325,진짜 작가님 이불속애 들어간 컷마다 발나온거 귀여워 미치겠음,0
326,나이먹어도 사람 안바뀌어요... ㅡ.ㅠ,0
327,씻기 전 : 피곤하니 씻고 자야지\n씻은 후 : 졸음이 싹 가시네,1
328,그 15분이 자꾸만 새끼를 치는 마법ㅜ,0
329,현실감 쩌는 복부 라인^^,0
330,독립일기만한 웹툰이 엄써요...데헷^^♥,0
331,ㅋㅋㅋㅋ ㅋㅋㅋㅋ 나랑 똑같애,0
332,진짜 등베개 광고 받으셔야해,1
333,지금이 그나마 겨울이라 괜찮긴 한데 자취생이 한여름에 설거지를 안하고 외출을 할 경...,0
334,새벽4시인가요? 난 그때 일어나는뎁 하는일이 시내버스라....,1


출처 : https://github.com/reniew/NSMC_Sentimental-Analysis/blob/master/notebook/NSMC_Preprocessing.ipynb  